# Assignment 1

In [1]:
#@title Mount your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !mkdir -p /content/gdrive/MyDrive/Projects
# cd /content/gdrive/MyDrive/Projects

In [42]:

!git pull origin main

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 552 bytes | 552.00 KiB/s, done.
From https://github.com/rasoulpanahi/IFT6135-2025-First-Assignment
 * branch            main       -> FETCH_HEAD
   a9d0357..7667e2a  main       -> origin/main
Updating a9d0357..7667e2a
Fast-forward
 HW1_2025/assignment1_release/mlpmixer.py | 4 +++-
 1 file changed, 3 insertions(+), 1 deletion(-)


In [5]:
# rm -rf IFT6135-2025-First-Assignment

# # Clone the repository again
!git clone https://github.com/rasoulpanahi/IFT6135-2025-First-Assignment.git

Cloning into 'IFT6135-2025-First-Assignment'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 59 (delta 23), reused 53 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (59/59), 420.38 KiB | 1.69 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [28]:
#!git clone https://github.com/rasoulpanahi/IFT6135-2025-First-Assignment.git

fatal: destination path 'IFT6135-2025-First-Assignment' already exists and is not an empty directory.


In [43]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
# If you run this notebook locally or on a cluster (i.e. not on Google Colab)
# you can delete this cell which is specific to Google Colab. You may also
# change the paths for data/logs in Arguments below.
import sys
import os
import shutil
import warnings

folder = "/content/drive/MyDrive/Projects/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release" #@param {type:"string"}
!ln -Ts "$folder" /content/assignment 2> /dev/null

# Add the assignment folder to Python path
if '/content/assignment' not in sys.path:
  sys.path.insert(0, '/content/assignment')

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
  warnings.warn('CUDA is not available.')

### Running on GPU
For this assignment, it will be necessary to run your experiments on GPU. To make sure the notebook is running on GPU, you can change the notebook settings with
* (EN) `Edit > Notebook Settings`
* (FR) `Modifier > Paramètres du notebook`


assignment  gdrive  IFT6135-2025-First-Assignment  sample_data


In [19]:

%cd /content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release

/content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release


In [44]:
%matplotlib inline
import warnings

from dataclasses import dataclass
import torch
from torch import optim
from torchvision.datasets import CIFAR10
from torchvision import transforms
from utils import seed_experiment, to_device, cross_entropy_loss, compute_accuracy
from config import get_config_parser
import json
from mlp import MLP
from resnet18 import ResNet18
from mlpmixer import MLPMixer
from tqdm import tqdm
from torch.utils.data import DataLoader
import time
import os

# Local Test
Before run the experiment, here are some local test cases you can run for sanity check

In [45]:
import unittest
import test
suite = unittest.TestLoader().loadTestsFromModule(test)
unittest.TextTestRunner(verbosity=2).run(suite)


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


<unittest.runner.TextTestResult run=0 errors=0 failures=0>

## Experiments

Below we define a few default arguments to get you started with your experiments. You are encouraged to modify the function `main_entry()`, as well as these arguments, to fit your needs (e.g. changing hyperparameters, the optimizer, adding regularizations).

In [47]:
@dataclass
class Arguments:
  # Data
  batch_size: int = 128
  # Model
  model: str = 'mlp'  # [mlp, resnet18, mlpmixer]
  model_config: str = "./model_configs/mlp.json" # path to model config json file

  # Optimization
  optimizer: str = 'adamw'  # [sgd, momentum, adam, adamw]
  epochs: int = 15
  lr: float = 1e-3
  momentum: float = 0.9
  weight_decay: float = 5e-4

  # Experiment
  logdir: str = '/content/assignment/logs'
  seed: int = 42

  # Miscellaneous
  device: str = 'cuda'
  visualize : bool = False
  print_every: int = 80

In [48]:
# Main code entry. Train the model and save the logs
from main import train, evaluate
def main_entry(args):
    # Check for the device
    if (args.device == "cuda") and not torch.cuda.is_available():
        warnings.warn(
            "CUDA is not available, make that your environment is "
            "running on GPU (e.g. in the Notebook Settings in Google Colab). "
            'Forcing device="cpu".'
        )
        args.device = "cpu"

    if args.device == "cpu":
        warnings.warn(
            "You are about to run on CPU, and might run out of memory "
            "shortly. You can try setting batch_size=1 to reduce memory usage."
        )

    # Seed the experiment, for repeatability
    seed_experiment(args.seed)

    test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.49139968, 0.48215841, 0.44653091], [0.24703223, 0.24348513, 0.26158784])
                                     ])
    # For training, we add some augmentation. Networks are too powerful and would overfit.
    train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                          transforms.RandomResizedCrop((32,32),scale=(0.8,1.0),ratio=(0.9,1.1)),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.49139968, 0.48215841, 0.44653091], [0.24703223, 0.24348513, 0.26158784])
                                        ])
    # Loading the training dataset. We need to split it into a training and validation part
    # We need to do a little trick because the validation set should not use the augmentation.
    train_dataset = CIFAR10(root='./data', train=True, transform=train_transform, download=True)
    val_dataset = CIFAR10(root='./data', train=True, transform=test_transform, download=True)
    train_set, _ = torch.utils.data.random_split(train_dataset, [45000, 5000])
    _, val_set = torch.utils.data.random_split(val_dataset, [45000, 5000])

    # Loading the test set
    test_set = CIFAR10(root='./data', train=False, transform=test_transform, download=True)

    # Load model
    print(f'Build model {args.model.upper()}...')
    if args.model_config is not None:
        print(f'Loading model config from {args.model_config}')
        with open(args.model_config) as f:
            model_config = json.load(f)
    else:
        raise ValueError('Please provide a model config json')
    print(f'########## {args.model.upper()} CONFIG ################')
    for key, val in model_config.items():
        print(f'{key}:\t{val}')
    print('############################################')
    model_cls = {'mlp': MLP, 'resnet18': ResNet18, 'mlpmixer': MLPMixer}[args.model]
    model = model_cls(**model_config)
    model.to(args.device)

    # Optimizer
    if args.optimizer == "adamw":
        optimizer = optim.AdamW(
            model.parameters(), lr=args.lr, weight_decay=args.weight_decay
        )
    elif args.optimizer == "adam":
        optimizer = optim.Adam(model.parameters(), lr=args.lr)
    elif args.optimizer == "sgd":
        optimizer = optim.SGD(
            model.parameters(), lr=args.lr, weight_decay=args.weight_decay
        )
    elif args.optimizer == "momentum":
        optimizer = optim.SGD(
            model.parameters(),
            lr=args.lr,
            momentum=args.momentum,
            weight_decay=args.weight_decay,
        )

    print(
        f"Initialized {args.model.upper()} model with {sum(p.numel() for p in model.parameters())} "
        f"total parameters, of which {sum(p.numel() for p in model.parameters() if p.requires_grad)} are learnable."
    )

    train_losses, valid_losses = [], []
    train_accs, valid_accs = [], []
    train_times, valid_times = [], []

    # We define a set of data loaders that we can use for various purposes later.
    train_dataloader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, drop_last=True, pin_memory=True, num_workers=4)
    valid_dataloader = DataLoader(val_set, batch_size=args.batch_size, shuffle=False, drop_last=False, num_workers=4)
    test_dataloader = DataLoader(test_set, batch_size=args.batch_size, shuffle=False, drop_last=False, num_workers=4)
    for epoch in range(args.epochs):
        tqdm.write(f"====== Epoch {epoch} ======>")
        loss, acc, wall_time = train(epoch, model, train_dataloader, optimizer,args)
        train_losses.append(loss)
        train_accs.append(acc)
        train_times.append(wall_time)

        loss, acc, wall_time = evaluate(epoch, model, valid_dataloader,args)
        valid_losses.append(loss)
        valid_accs.append(acc)
        valid_times.append(wall_time)

    test_loss, test_acc, test_time = evaluate(
        epoch, model, test_dataloader, args, mode="test"
    )
    print(f"===== Best validation Accuracy: {max(valid_accs):.3f} =====>")

    # Save log if logdir provided
    if args.logdir is not None:
        print(f'Writing training logs to {args.logdir}...')
        os.makedirs(args.logdir, exist_ok=True)
        with open(os.path.join(args.logdir, 'results.json'), 'w') as f:
            f.write(json.dumps(
                {
                    "train_losses": train_losses,
                    "valid_losses": valid_losses,
                    "train_accs": train_accs,
                    "valid_accs": valid_accs,
                    "test_loss": test_loss,
                    "test_acc": test_acc
                },
                indent=4,
            ))

        # Visualize
        if args.visualize and args.model in ['resnet18', 'mlpmixer']:
            model.visualize(args.logdir)


In [16]:
# Example to run MLP with 15 epochs
config = Arguments(model='mlp',
                   model_config='/content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release/model_configs/mlp.json',
                   epochs=2, logdir="exps/mlp_default")
main_entry(config)

100%|██████████| 170M/170M [00:06<00:00, 25.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Build model MLP...
Loading model config from /content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release/model_configs/mlp.json
########## MLP CONFIG ################
input_size:	3072
hidden_sizes:	[1024, 512, 64, 64]
num_classes:	10
activation:	relu
############################################
Initialized MLP model with 3709194 total parameters, of which 3709194 are learnable.
====== Epoch 0 ======>


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[TRAIN] Epoch: 0, Iter: 0, Loss: 2.61241
[TRAIN] Epoch: 0, Iter: 80, Loss: 1.77236
[TRAIN] Epoch: 0, Iter: 160, Loss: 1.74864
[TRAIN] Epoch: 0, Iter: 240, Loss: 1.54849
[TRAIN] Epoch: 0, Iter: 320, Loss: 1.66133
== [TRAIN] Epoch: 0, Accuracy: 0.358 ==>
[VAL] Epoch: 0, Iter: 0, Loss: 1.69828
=== [VAL] Epoch: 0, Iter: 39, Accuracy: 0.413 ===>
====== Epoch 1 ======>
[TRAIN] Epoch: 1, Iter: 0, Loss: 1.72243
[TRAIN] Epoch: 1, Iter: 80, Loss: 1.56231
[TRAIN] Epoch: 1, Iter: 160, Loss: 1.62889
[TRAIN] Epoch: 1, Iter: 240, Loss: 1.76265
[TRAIN] Epoch: 1, Iter: 320, Loss: 1.73134
== [TRAIN] Epoch: 1, Accuracy: 0.430 ==>
[VAL] Epoch: 1, Iter: 0, Loss: 1.57578
=== [VAL] Epoch: 1, Iter: 39, Accuracy: 0.471 ===>
[TEST] Epoch: 1, Iter: 0, Loss: 1.37968
=== [TEST] Epoch: 1, Iter: 78, Accuracy: 0.461 ===>
===== Best validation Accuracy: 0.471 =====>
Writing training logs to exps/mlp_default...


In [40]:
# Example to run MLP with 15 epochs
config = Arguments(model='resnet18',
                   model_config='/content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release/model_configs/resnet18.json',
                   epochs=2, logdir="exps/resnet18")
main_entry(config)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Build model RESNET18...
Loading model config from /content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release/model_configs/resnet18.json
########## RESNET18 CONFIG ################
num_classes:	10
############################################
Initialized RESNET18 model with 11173962 total parameters, of which 11173962 are learnable.
====== Epoch 0 ======>


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[TRAIN] Epoch: 0, Iter: 0, Loss: 2.35414
[TRAIN] Epoch: 0, Iter: 80, Loss: 1.51107
[TRAIN] Epoch: 0, Iter: 160, Loss: 1.43727
[TRAIN] Epoch: 0, Iter: 240, Loss: 1.30898
[TRAIN] Epoch: 0, Iter: 320, Loss: 1.21086
== [TRAIN] Epoch: 0, Accuracy: 0.473 ==>
[VAL] Epoch: 0, Iter: 0, Loss: 1.41436
=== [VAL] Epoch: 0, Iter: 39, Accuracy: 0.570 ===>
====== Epoch 1 ======>
[TRAIN] Epoch: 1, Iter: 0, Loss: 1.03957
[TRAIN] Epoch: 1, Iter: 80, Loss: 1.02246
[TRAIN] Epoch: 1, Iter: 160, Loss: 0.96548
[TRAIN] Epoch: 1, Iter: 240, Loss: 0.92622
[TRAIN] Epoch: 1, Iter: 320, Loss: 0.76036
== [TRAIN] Epoch: 1, Accuracy: 0.657 ==>
[VAL] Epoch: 1, Iter: 0, Loss: 0.92165
=== [VAL] Epoch: 1, Iter: 39, Accuracy: 0.671 ===>
[TEST] Epoch: 1, Iter: 0, Loss: 0.88246
=== [TEST] Epoch: 1, Iter: 78, Accuracy: 0.655 ===>
===== Best validation Accuracy: 0.671 =====>
Writing training logs to exps/resnet18...


In [49]:
# Example to run MLP with 15 epochs
config = Arguments(model='mlpmixer',
                   model_config='/content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release/model_configs/mlpmixer.json',
                   epochs=2, logdir="exps/mlpmixer")
main_entry(config)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Build model MLPMIXER...
Loading model config from /content/IFT6135-2025-First-Assignment/HW1_2025/assignment1_release/model_configs/mlpmixer.json
########## MLPMIXER CONFIG ################
num_classes:	10
img_size:	32
patch_size:	4
embed_dim:	256
num_blocks:	4
drop_rate:	0.0
activation:	gelu
############################################
Initialized MLPMIXER model with 2188298 total parameters, of which 2188298 are learnable.
====== Epoch 0 ======>


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[TRAIN] Epoch: 0, Iter: 0, Loss: 2.31901
[TRAIN] Epoch: 0, Iter: 80, Loss: 1.57623
[TRAIN] Epoch: 0, Iter: 160, Loss: 1.61766
[TRAIN] Epoch: 0, Iter: 240, Loss: 1.41238
[TRAIN] Epoch: 0, Iter: 320, Loss: 1.47204
== [TRAIN] Epoch: 0, Accuracy: 0.421 ==>
[VAL] Epoch: 0, Iter: 0, Loss: 1.39669
=== [VAL] Epoch: 0, Iter: 39, Accuracy: 0.522 ===>
====== Epoch 1 ======>
[TRAIN] Epoch: 1, Iter: 0, Loss: 1.21684
[TRAIN] Epoch: 1, Iter: 80, Loss: 1.32808
[TRAIN] Epoch: 1, Iter: 160, Loss: 1.19022
[TRAIN] Epoch: 1, Iter: 240, Loss: 1.21163
[TRAIN] Epoch: 1, Iter: 320, Loss: 1.03568
== [TRAIN] Epoch: 1, Accuracy: 0.561 ==>
[VAL] Epoch: 1, Iter: 0, Loss: 1.21091
=== [VAL] Epoch: 1, Iter: 39, Accuracy: 0.610 ===>
[TEST] Epoch: 1, Iter: 0, Loss: 1.00693
=== [TEST] Epoch: 1, Iter: 78, Accuracy: 0.599 ===>
===== Best validation Accuracy: 0.610 =====>
Writing training logs to exps/mlpmixer...
